Let's try some basic tests of `VB.jl`:

In [1]:
include("VB.jl")
using VB
using Distributions

# Make a factor and check its value

In [2]:
μ = rand()
τ = rand()
x = Normal(μ, τ)
f = LogNormalFactor(x, μ, τ)

VB.LogNormalFactor(Distributions.Normal(μ=0.08413732987630462, σ=0.051986794368444667),0.08413732987630462,0.051986794368444667)

In [3]:
value(f)

0.5593739898965625

In [4]:
g = EntropyFactor(x)
value(g)

-1.537827013900917

In [5]:
naturals(x)

(31.131686486597527,-185.00519645896833)

# Let's make a simple model

We want a simple model of inference for a normal distribution:

$$
\begin{aligned}
    y \sim \mathcal{N}(\mu, \tau) \\
    \mu \sim \mathcal{N}(\mu_0, \tau_0) \\
    \tau \sim \mathrm{Ga}(\alpha, \beta) \\
    q(\mu) = \mathcal{N}(m, t) \\
    q(\tau) = \mathrm{Ga}(a, b)
\end{aligned}
$$

This model is conjugate, and can be solved by straightforward updates of the natural parameters of the posterior.

## Make the nodes

We need to make nodes for all the random variables that will need to be updated (we can also create nodes for parameter arrays, but these will be converted automatically if we don't).

In [12]:
dims = (5, 6)

# note: it won't matter much how we initialize here
μ = NodeArray(Normal, zeros(dims), ones(dims))
τ = NodeArray(Gamma, ones(dims), ones(dims))

nobs = 20
y = rand(nobs, dims...);

Now make factors: We need a Normal factor for the observation model plus a prior and an entropy for each node.

In [13]:
obs = LogNormalFactor(y, μ, τ)
μ_prior = LogNormalFactor(μ, zeros(dims), 2 * ones(dims))
τ_prior = LogGammaFactor(τ, 1.1 * ones(dims), ones(dims))

VB.LogGammaFactor(5x6 Array{Distributions.Gamma,2}:
 Distributions.Gamma(α=1.0, θ=1.0)  …  Distributions.Gamma(α=1.0, θ=1.0)
 Distributions.Gamma(α=1.0, θ=1.0)     Distributions.Gamma(α=1.0, θ=1.0)
 Distributions.Gamma(α=1.0, θ=1.0)     Distributions.Gamma(α=1.0, θ=1.0)
 Distributions.Gamma(α=1.0, θ=1.0)     Distributions.Gamma(α=1.0, θ=1.0)
 Distributions.Gamma(α=1.0, θ=1.0)     Distributions.Gamma(α=1.0, θ=1.0),5x6 Array{Float64,2}:
 1.1  1.1  1.1  1.1  1.1  1.1
 1.1  1.1  1.1  1.1  1.1  1.1
 1.1  1.1  1.1  1.1  1.1  1.1
 1.1  1.1  1.1  1.1  1.1  1.1
 1.1  1.1  1.1  1.1  1.1  1.1,5x6 Array{Float64,2}:
 1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0)

In [21]:
value(obs), value(μ_prior), value(τ_prior)

(-38.35125483195037,-67.96536370453937,7.496173237795199)

In [22]:
m = VBModel(Dict(:μ => μ, :τ => τ), [obs, μ_prior, τ_prior])

VB.VBModel(Dict{Symbol,VB.Node}(:τ=>VB.RandomNode{Distributions.Gamma,2}(5x6 Array{Distributions.Gamma,2}:
 Distributions.Gamma(α=1.0, θ=1.0)  …  Distributions.Gamma(α=1.0, θ=1.0)
 Distributions.Gamma(α=1.0, θ=1.0)     Distributions.Gamma(α=1.0, θ=1.0)
 Distributions.Gamma(α=1.0, θ=1.0)     Distributions.Gamma(α=1.0, θ=1.0)
 Distributions.Gamma(α=1.0, θ=1.0)     Distributions.Gamma(α=1.0, θ=1.0)
 Distributions.Gamma(α=1.0, θ=1.0)     Distributions.Gamma(α=1.0, θ=1.0),Dict{VB.Factor,Symbol}(VB.LogGammaFactor(VB.RandomNode{Distributions.Gamma,2}(#= circular reference =#),VB.ConstantNode{Float64,2}(5x6 Array{Float64,2}:
 1.1  1.1  1.1  1.1  1.1  1.1
 1.1  1.1  1.1  1.1  1.1  1.1
 1.1  1.1  1.1  1.1  1.1  1.1
 1.1  1.1  1.1  1.1  1.1  1.1
 1.1  1.1  1.1  1.1  1.1  1.1,Dict{VB.Factor,Symbol}()),VB.ConstantNode{Float64,2}(5x6 Array{Float64,2}:
 1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0,

In [23]:
m.nodes[:μ].factormap

Dict{VB.Factor,Symbol} with 2 entries:
  VB.LogNormalFactor(VB.C… => :μ
  VB.LogNormalFactor(VB.R… => :x

In [24]:
naturals(obs, Val{:x}, μ)

(
5x6 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0,

5x6 Array{Float64,2}:
 -0.5  -0.5  -0.5  -0.5  -0.5  -0.5
 -0.5  -0.5  -0.5  -0.5  -0.5  -0.5
 -0.5  -0.5  -0.5  -0.5  -0.5  -0.5
 -0.5  -0.5  -0.5  -0.5  -0.5  -0.5
 -0.5  -0.5  -0.5  -0.5  -0.5  -0.5)

In [25]:
naturals(obs, Val{:μ}, μ)

(
5x6 Array{Float64,2}:
 0.019526  0.64192   0.920968   0.221961  0.497111  0.376147
 0.645838  0.886014  0.635094   0.86881   0.173711  0.234695
 0.571483  0.605633  0.240604   0.252273  0.424832  0.433595
 0.246642  0.580497  0.81434    0.503185  0.399902  0.499845
 0.8564    0.315123  0.0795404  0.555742  0.340429  0.753323,

5x6 Array{Float64,2}:
 -0.5  -0.5  -0.5  -0.5  -0.5  -0.5
 -0.5  -0.5  -0.5  -0.5  -0.5  -0.5
 -0.5  -0.5  -0.5  -0.5  -0.5  -0.5
 -0.5  -0.5  -0.5  -0.5  -0.5  -0.5
 -0.5  -0.5  -0.5  -0.5  -0.5  -0.5)

In [26]:
naturals(obs, Val{:τ}, τ)

(0.5,
5x6 Array{Float64,2}:
 0.500191  0.706031  0.924091  0.524633  0.62356   0.570743
 0.708553  0.89251   0.701672  0.877416  0.515088  0.527541
 0.663296  0.683396  0.528945  0.531821  0.590241  0.594002
 0.530416  0.668488  0.831575  0.626598  0.579961  0.624922
 0.86671   0.549651  0.503163  0.654425  0.557946  0.783748)

In [27]:
method_exists(naturals, (LogNormalFactor, Type{Val{:x}}, RandomNode{Normal}))

true

In [28]:
check_conjugate(m.nodes[:τ])

true

In [29]:
μ.data

5x6 Array{Distributions.Normal,2}:
 Distributions.Normal(μ=0.0, σ=1.0)  …  Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)     Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)     Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)     Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)     Distributions.Normal(μ=0.0, σ=1.0)

In [30]:
update!(μ, Val{:conjugate})

In [31]:
μ.data

5x6 Array{Distributions.Normal,2}:
 Distributions.Normal(μ=-0.33974642587876164, σ=0.7071067811865476)  …  Distributions.Normal(μ=-0.08757735964789944, σ=0.7071067811865476)   
 Distributions.Normal(μ=0.1031230083822061, σ=0.7071067811865476)       Distributions.Normal(μ=-0.18759892024976002, σ=0.7071067811865476)   
 Distributions.Normal(μ=0.05054604661844582, σ=0.7071067811865476)      Distributions.Normal(μ=-0.046955363717725414, σ=0.7071067811865476)  
 Distributions.Normal(μ=-0.17915149839257358, σ=0.7071067811865476)     Distributions.Normal(μ=-0.00010971345274074199, σ=0.7071067811865476)
 Distributions.Normal(μ=0.2520128176967853, σ=0.7071067811865476)       Distributions.Normal(μ=0.17912620274862429, σ=0.7071067811865476)    